# Polymers

This looks nasty, because the most obvious solution here is to build a map function that takes each pairwise letter and produces a new combination, and then combines all the resultant combinations, but I have a feeling that this is going to expand into memory hogging space really quickly.  There's something about the question here that makes me wonder if there's an easier way to calculate teh totals, but I can't think of it yet.

So here we go with the version that is going to grow a lot over just 10 times.

Start with a string, and use a lookup table.

The example lookup table says something like "CN->C", but I suspect that we should think about the substitutions we want to make, so CN->CCN.  That's a complete substitution.

However, as we run over something like NNCB, if we turn NN->NCN and NC->NBC and CB->CHB then we're going to end up with NCNNBCCHB with duplicated letters.  Instead if we crop the first letter, we end up with NN->CN, NC->BC and CB->HB, we end up with "N" plus CNBCHB.

That's a much simpler mapping, and we can turn the initial table into that lookup form fairly easily I hope.  Let's give that a try

In [1]:
## Import ipytest and get it setup for use in Python Notebook
import pytest
import ipytest
ipytest.autoconfig()

In [2]:
test_lines = """NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C""".split("\n")

def parse_lines(lines):
    template = lines[0]
    mapping = {}
    for line in lines[2:]:
        start, append = line.split(" -> ")
        mapping[start] = append+start[1]
    return template, mapping


template, mapping = parse_lines(test_lines)
assert template == "NNCB"
assert mapping["CH"] == "BH"
assert mapping["HH"] == "NH"
assert mapping["CB"] == "HB"
assert mapping["NH"] == "CH"
assert len(mapping) == 16

Ok, let's try some expansion, so each iteration is equal to the pairwise mapping of letters, looked up in the substituion chart.  Luckily python has us already covered for pairwise iteration of a collection, in itertools with the well named "pairwise" function

In [3]:
import itertools
import collections

def iterate(polymer, mapping):
    l = [polymer[0]]
    for pair in itertools.pairwise(polymer):
        l.append(mapping["".join(pair)])
    return "".join(l)

assert iterate(template, mapping) == "NCNBCHB"
assert iterate("NCNBCHB", mapping) == "NBCCNBBBCBHCB"
assert iterate("NBCCNBBBCBHCB", mapping) == "NBBBCNCCNBBNBNBBCHBHHBCHB"
assert iterate("NBBBCNCCNBBNBNBBCHBHHBCHB", mapping) == "NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB"

polymer = template
for i in range(10):
    polymer = iterate(polymer, mapping)

assert 3073 == len(polymer)
counts = collections.Counter(polymer).most_common()
print(counts[0],counts[-1])
print(counts[0][1]-counts[-1][1])

('B', 1749) ('H', 161)
1588


Let's do that on production data... gulp

In [4]:
polymer, mapping = parse_lines([line.strip() for line in open("day14.txt").readlines()])
for i in range(10):
    polymer = iterate(polymer, mapping)

print(len(polymer))
counts = collections.Counter(polymer).most_common()
print(counts[0],counts[-1])
print(counts[0][1]-counts[-1][1])

19457
('B', 3330) ('H', 533)
2797


# More steps

As I thought, 30 more steps.  I suspect this doubles in complexity with each step taken.

More importantly, if there are in fact around 2 trillion "B"'s in the string, then we're going to need around 2TB of ram to just hold the "B"'s in bytes, so that's not going to work.

We're going to need a different data structure and different algorithm, and I've not had the week to think about that I'm afraid.